## LSTM_NowTV
## Table of Contents:
* [0. Importing dependencies](#dependencies)
* [1. Setting up of the dataset](#1.0)
    * [1.1 Importing of the dataset](#1.1)
    * [1.2 Splitting of the dataset](#1.2)
* [2. Model Implementation](#2.0)
    * [2.2 Model training](#2.1)


This LSTM model implements multi-task output and predicts future 24-hour viewership by applying a rolling window.

Although the results from the first five epochs suggest that this optimized LSTM model has surpassed last year's model in terms of prediction accuracy, it has proven to be inefficient whether running on Google Colab or on the company's remote server. This inefficiency is believed to be due to the high computational demand caused by the excessively fine granularity and large volume of data. Therefore, given the current computing resources available, the project has decided to temporarily abandon the use of the LSTM model as a forecasting model.

Nonetheless, this file has been uploaded to the repository for record and reference.

# 0. Importing dependencies  <a class="anchor" id="dependencies"></a>

In [ ]:
import pandas as pd

import numpy as np

from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf

# 1. Setting up of the dataset <a class="anchor" id="1.0"></a>

## 1.1 Importing of the dataset <a class="anchor" id="1.1"></a>

In [ ]:
# Load the final dataframe
file_path = '/content/gdrive/My Drive/df_final.csv'
df_final = pd.read_csv(file_path)

print(df_final.columns)

Index(['trend_vc', 'yearly_vc', 'weekly_vc', 'trend_i', 'yearly_i', 'weekly_i',
       'trend_ud', 'yearly_ud', 'weekly_ud', 'videoConsumption', 'impression'],
      dtype='object')


## 1.2 Splitting the dataset <a class="anchor" id="1.2"></a>

In [ ]:
def split_dataset(df):
    # Divide the data into 70% for the training set, 15% for the validation set, and 15% for the test set
    idx_train_end = round(0.7 * len(df))
    idx_val_end = round(0.85 * len(df))

    # Split the DataFrame into training, validation, and test sets
    df_train = df.iloc[:idx_train_end]
    df_val = df.iloc[idx_train_end:idx_val_end]
    df_test = df.iloc[idx_val_end:]

    return df_train, df_val, df_test

# Conduct dataset splitting on df_final
df_train, df_val, df_test = split_dataset(df_final)

#2. Model Implementation <a class="anchor" id="2.0"></a>

## 2.1 Model training <a class="anchor" id="2.1"></a>

In [ ]:
# Define input features and target outputs
features = ['weekly_vc', 'weekly_i', 'weekly_ud']
targets = ['videoConsumption', 'impression']
number_of_features = len(features)

# Selecting features and target columns from
train_features = df_train[features + targets]

scaler = MinMaxScaler(feature_range=(0,1))
train_scaled = scaler.fit_transform(train_features)

# Convert the scaled array back to a DataFrame, keeping the column names
train_scaled = pd.DataFrame(train_scaled, columns=features + targets)

# Same for val and test set
val_features = df_val[features + targets]
val_scaled = scaler.transform(val_features)
val_scaled = pd.DataFrame(val_scaled, columns=features + targets)

test_features = df_test[features + targets]
test_scaled = scaler.transform(test_features)
test_scaled = pd.DataFrame(test_scaled, columns=features + targets)

# Obtain train_data, val_data and test_data
train_data = train_scaled.values
val_data = val_scaled.values
test_data = test_scaled.values


In [ ]:
def rolling_window_generator(data, window_size, forecast_horizon, batch_size=32):
    total_size = len(data) - window_size - forecast_horizon + 1
    num_batches = total_size // batch_size

    while True:
        for i in range(num_batches):
            start = i * batch_size
            end = start + batch_size
            batch_X, batch_y_vc, batch_y_i = [], [], []
            for j in range(start, end):
                batch_X.append(data[j:(j + window_size), :-2])  # Features
                # Two dimension outputs
                batch_y_vc.append(data[(j + window_size):(j + window_size + forecast_horizon), -2])
                batch_y_i.append(data[(j + window_size):(j + window_size + forecast_horizon), -1])
            yield np.array(batch_X), [np.array(batch_y_vc), np.array(batch_y_i)]


In [ ]:
# Create Evaluation Metric
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

def mape(y_true, y_pred):
    return tf.reduce_mean(tf.abs((y_true - y_pred) / y_true)) * 100


In [ ]:
# Set h values
h = 1440  # 1-Day 1440, 7-Day 10080, 30-Day 43920
# number_of_features = 3  # 'weekly_vc', 'weekly_i', 'weekly_ud'

input_layer = Input(shape=(h, number_of_features))
lstm_layer = LSTM(50, return_sequences=False)(input_layer)

# Define two output layers
output_vc = Dense(1440, activation='linear', name='videoConsumption')(lstm_layer)
output_i = Dense(1440, activation='linear', name='impression')(lstm_layer)

# Initialize model
model = Model(inputs=input_layer, outputs=[output_vc, output_i])
model.compile(optimizer='adam',
              loss={'videoConsumption': 'mean_squared_error', 'impression': 'mean_squared_error'},
              metrics={'videoConsumption': ['mse', rmse, mape], 'impression': ['mse', rmse, mape]})

# Apply rolling window
train_gen = rolling_window_generator(train_data, h, 1440, 32)
val_gen = rolling_window_generator(val_data, h, 1440, 32)

# Calculate steps_per_epoch
train_steps = (len(train_data) - h - 1440 + 1) // 32
val_steps = (len(val_data) - h - 1440 + 1) // 32

# Train model
history = model.fit(train_gen, steps_per_epoch=train_steps, epochs=7, validation_data=val_gen, validation_steps=val_steps)

# Save model
model_save_path = '/content/drive/My Drive/lstm_NowTV_1440.h5'
model.save(model_save_path)

# Plotting training and validation losses
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['videoConsumption_loss'], label='Train Loss for Video Consumption')
plt.plot(history.history['val_videoConsumption_loss'], label='Validation Loss for Video Consumption')
plt.title('Video Consumption Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['impression_loss'], label='Train Loss for Impression')
plt.plot(history.history['val_impression_loss'], label='Validation Loss for Impression')
plt.title('Impression Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

# Model Evaluation
loss_vc, mse_vc, rmse_vc, mape_vc, loss_i, mse_i, rmse_i, mape_i = model.evaluate(val_gen, steps=val_steps)
print(f"Validation loss for Video Consumption (vc)={loss_vc}, RMSE={rmse_vc}, MAPE={mape_vc}")
print(f"Validation loss for Impression (i)={loss_i}, RMSE={rmse_i}, MAPE={mape_i}")


Epoch 1/7
34002/34002 [==============================] - 17957s 528ms/step - loss: 0.0160 - videoConsumption_loss: 0.0134 - impression_loss: 0.0026 - videoConsumption_mse: 0.0134 - videoConsumption_rmse: 0.1072 - videoConsumption_mape: inf - impression_mse: 0.0026 - impression_rmse: 0.0493 - impression_mape: inf - val_loss: 0.0159 - val_videoConsumption_loss: 0.0135 - val_impression_loss: 0.0023 - val_videoConsumption_mse: 0.0135 - val_videoConsumption_rmse: 0.1055 - val_videoConsumption_mape: 332.0480 - val_impression_mse: 0.0023 - val_impression_rmse: 0.0461 - val_impression_mape: inf
Epoch 2/7
34002/34002 [==============================] - 17853s 525ms/step - loss: 0.0170 - videoConsumption_loss: 0.0142 - impression_loss: 0.0028 - videoConsumption_mse: 0.0142 - videoConsumption_rmse: 0.1045 - videoConsumption_mape: inf - impression_mse: 0.0028 - impression_rmse: 0.0480 - impression_mape: inf - val_loss: 0.0152 - val_videoConsumption_loss: 0.0130 - val_impression_loss: 0.0022 - val_v